## Access this Notebook
You can launch this notebook in the US GHG Center JupyterHub by clicking the link below. If you are a new user, you should first sign up for the hub by filling out this [**request form**](https://docs.google.com/forms/d/e/1FAIpQLSdai8otCdrVQzJgev8mjDhzKyCg7jcrB3UeTXNHoCiaMKrkaQ/viewform) and providing the required information. 


Access the [**EMIT Methane Point Source Plume Complexes**](https://us-ghg-center.github.io/ghgc-docs/user_data_notebooks/emit-ch4plume-v1_User_Notebook.html) notebook in the US GHG Center JupyterHub. 

## Table of Contents
- [Data Summary and Application](#data-summary-and-application)
- [Approach](#approach)
- [About the Data](#about-the-data)
- [Install the Required Libraries](#install-the-required-libraries)
- [Query the STAC API](#query-the-stac-api)
- [Visual Comparison Across Time Periods](#visual-comparison-across-time-periods)
- [Map Out Selected Tiles](#map-out-selected-tiles)
- [Summary](#summary)

## Data Summary and Application
- **Spatial coverage**: 52°N to 52°S latitude within target mask
- **Spatial resolution**: 60 m
- **Temporal extent**: August 1, 2022 - Ongoing
- **Temporal resolution**: Variable (based on ISS orbit, solar illumination, and target mask)
- **Unit**: Parts per million meter (ppm-m)
- **Utility**: Methane Emissions, Plume Detection, Climate Monitoring 

For more, visit the [EMIT Methane Point Source Plume Complexes](https://earth.gov/ghgcenter/data-catalog/emit-ch4plume-v1) data overview page.  

## Approach

1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. The collection processed in this notebook is the Earth Surface Mineral Dust Source Investigation (EMIT) methane emission plumes data product.
2. Pass the STAC item into the raster API `/collections/{collection_id}/items/{item_id}/tilejson.json` endpoint.
3. Using `folium.Map`, visualize the plumes.
4. After the visualization, perform zonal statistics for a given polygon.

   

## About the Data
#### EMIT Methane Point Source Plume Complexes
The Earth Surface Mineral Dust Source Investigation (EMIT) instrument builds upon NASA’s long history of developing advanced imaging spectrometers for new science and applications. EMIT launched to the International Space Station (ISS) on July 14, 2022. The data shows high-confidence research grade methane plumes from point source emitters - updated as they are identified - in keeping with Jet Propulsion Laboratory (JPL) Open Science and Open Data policy. 

Large methane emissions, typically referred to as point source emissions, represent a significant proportion of total methane emissions from the production, transport, and processing of oil and natural gas, landfills, and other sources. By measuring the spectral fingerprint of methane, EMIT can map areas of high methane concentration over background levels in the atmosphere, identifying plume complexes, and estimating the methane enhancements.

For more information regarding this dataset, please visit the [EMIT Methane Point Source Plume Complexes](https://earth.gov/ghgcenter/data-catalog/emit-ch4plume-v1) data overview page.

## Terminology
Navigating data via the GHGC API, you will encounter terminology that is different from browsing in a typical filesystem. We'll define some terms here which are used throughout this notebook.
- `catalog`:    All datasets available at the `/stac` endpoint
- `collection`: A specific dataset, e.g. EMIT Methane Point Source Plume Complexes
- `item`:       One granule in the dataset, e.g. one file of EMIT data
- `asset`:      A variable available within the granule, e.g. CH4 plume emissions
- `STAC API`:   **Sp**atio**T**emporal **A**sset **C**atalogs - Endpoint for fetching metadata about available datasets
- `Raster API`: Endpoint for fetching data itself, for imagery and statistics

# Install the Required Libraries
Required libraries are pre-installed on the GHG Center Hub, except the `tabulate` and `seaborn` libraries. If you need to run this notebook elsewhere, please install the libraries by running the following command line:

%pip install requests folium rasterstats pystac_client pandas matplotlib --quiet

In [20]:
# Import the following libraries
# For fetching from the Raster API
import requests
# For making maps
import folium
import folium.plugins
from folium import Map, TileLayer
# For talking to the STAC API
from pystac_client import Client
# For working with data
import pandas as pd
# For making time series
import matplotlib.pyplot as plt
# For formatting date/time data
import datetime
# Custom functions for working with GHGC data via the API
import ghgc_utils

## Query the STAC API
First, we are going to import the required libraries. Once imported, they allow better executing a query in the GHG Center Spatio Temporal Asset Catalog (STAC) Application Programming Interface (API) where the granules for this collection are stored.

**STAC API Collection Names**

Now, you must fetch the dataset from the [**STAC API**](https://earth.gov/ghgcenter/api/stac/) by defining its associated STAC API collection ID as a variable. 
The collection ID, also known as the **collection name**, for the EMIT Methane Point Source Plume Complexes dataset is [**emit-ch4plume-v1**](https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1)

In [2]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for methane emission plumes 
collection_name = "emit-ch4plume-v1"

In [3]:
# Fetch the collection from the STAC API using the appropriate endpoint
# The 'pystac_client' library allows a HTTP request possible
catalog = Client.open(STAC_API_URL)
collection = catalog.get_collection(collection_name)

# Print the properties of the collection to the console
collection

<CollectionClient id=emit-ch4plume-v1>

In [4]:
# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for methane emission plumes 
collection_name = "emit-ch4plume-v1"

In [5]:
# Next, you need to specify the asset name for this collection.
# The asset name refers to the raster band containing the pixel values for the parameter of interest.
# For the case of the EMIT Methane Point Source collection, the parameter of interest is “ch4-plume-emissions”.
asset_name = "ch4-plume-emissions"

Examining the contents of our `collection` under the `temporal` variable, we note that data is available from August 2022 to May 2023. By looking at the `dashboard: time density`, we can see that observations are conducted daily and non-periodically (i.e., there are plumes emissions for multiple places on the same dates).

In [6]:
items = list(collection.get_items())  # Convert the iterator to a list
print(f"Found {len(items)} items")

Found 1574 items


In [7]:
search = catalog.search(
    collections=collection_name,
    datetime=['2023-01-01T00:00:00Z','2023-12-31T00:00:00Z']
)
# Take a look at the items we found
for item in search.item_collection():
    print(item)

<Item id=EMIT_L2B_CH4PLM_001_20231226T180707_002296>
<Item id=EMIT_L2B_CH4PLM_001_20231226T082757_002299>
<Item id=EMIT_L2B_CH4PLM_001_20231226T082757_002298>
<Item id=EMIT_L2B_CH4PLM_001_20231226T065514_002303>
<Item id=EMIT_L2B_CH4PLM_001_20231226T065450_002302>
<Item id=EMIT_L2B_CH4PLM_001_20231226T065426_002301>
<Item id=EMIT_L2B_CH4PLM_001_20231225T043707_002351>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174545_002264>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174545_002261>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174533_002613>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174533_002267>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174533_002266>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174533_002259>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174533_002255>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174533_002250>
<Item id=EMIT_L2B_CH4PLM_001_20231224T174533_002249>
<Item id=EMIT_L2B_CH4PLM_001_20231224T065746_002284>
<Item id=EMIT_L2B_CH4PLM_001_20231224T065746_002283>
<Item id=EMIT_L2B_CH4PLM_001_20231223T123006_0

In [9]:
# Examine the first item in the collection
# Keep in mind that a list starts from 0, 1, 2... therefore items[0] is referring to the first item in the list/collection
items[0]

<Item id=EMIT_L2B_CH4PLM_001_20241026T172133_003687>

## Map Out Selected Tiles

You will now explore global methane emission plumes from point sources and visualize the results on a map using folium. 

In [10]:
# To access the year value from each item more easily, this will let us query more explicitly by year and month (e.g., 2020-02)
items = {item.properties["datetime"][:10]: item for item in collection.get_items()}

In [11]:
items

{'2024-10-26': <Item id=EMIT_L2B_CH4PLM_001_20241026T061152_003688>,
 '2024-10-20': <Item id=EMIT_L2B_CH4PLM_001_20241020T092144_003679>,
 '2024-10-19': <Item id=EMIT_L2B_CH4PLM_001_20241019T100941_003675>,
 '2024-10-17': <Item id=EMIT_L2B_CH4PLM_001_20241017T083946_003674>,
 '2024-10-15': <Item id=EMIT_L2B_CH4PLM_001_20241015T114923_003697>,
 '2024-10-14': <Item id=EMIT_L2B_CH4PLM_001_20241014T201756_003690>,
 '2024-10-12': <Item id=EMIT_L2B_CH4PLM_001_20241012T123436_003668>,
 '2024-10-11': <Item id=EMIT_L2B_CH4PLM_001_20241011T223625_003666>,
 '2024-10-10': <Item id=EMIT_L2B_CH4PLM_001_20241010T201028_003656>,
 '2024-10-07': <Item id=EMIT_L2B_CH4PLM_001_20241007T082015_003686>,
 '2024-10-06': <Item id=EMIT_L2B_CH4PLM_001_20241006T090738_003685>,
 '2024-10-05': <Item id=EMIT_L2B_CH4PLM_001_20241005T095454_003683>,
 '2024-10-04': <Item id=EMIT_L2B_CH4PLM_001_20241004T025747_003647>,
 '2024-10-03': <Item id=EMIT_L2B_CH4PLM_001_20241003T065107_003631>,
 '2024-10-01': <Item id=EMIT_L2B_C

In [12]:
# To change the year and month of the observed parameter, you can modify the "items['YYYY-MM']" statement
# For example, you can change the current statement "items['2003-12']" to "items['2016-10']" 

observation_date_1 = items["2024-10-26"]

# Extract collection name and item ID
collection_id = observation_date_1.collection_id
item_id = observation_date_1.id

In [34]:
object = observation_date_1.assets[asset_name]
raster_bands = object.extra_fields.get("raster:bands", [{}])
rescale_values = {
    "max": raster_bands[0].get("histogram", {}).get("max"),
    "min": -1*raster_bands[0].get("histogram", {}).get("max"),
}

print(rescale_values)
print(raster_bands)

{'max': 1805.1678466796875, 'min': -1805.1678466796875}
[{'scale': 1.0, 'nodata': -9999.0, 'offset': 0.0, 'sampling': 'area', 'data_type': 'float32', 'histogram': {'max': 1805.1678466796875, 'min': -931.218505859375, 'count': 11, 'buckets': [17, 93, 276, 473, 352, 227, 95, 32, 6, 5]}, 'statistics': {'mean': 159.29984930203045, 'stddev': 395.47573187031026, 'maximum': 1805.1678466796875, 'minimum': -931.218505859375, 'valid_percent': 79.5959595959596}}]


Now, you will pass the item id, collection name, asset name, and the `rescaling factor` to the `Raster API` endpoint.

In [42]:
# Choose a color for displaying the tiles
# Please refer to matplotlib library if you'd prefer choosing a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html
color_map = "PuOr_r"

# Make a GET request to retrieve information for the date mentioned below
observation_date_1_tile = requests.get(
    f"{RASTER_API_URL}/collections/{collection_id}/items/{item_id}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map.lower()}"
    f"&rescale=0,1500"
).json()

# Print the properties of the retrieved granule to the console
observation_date_1_tile


{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20230418T060602_000822/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=puor_r&rescale=0%2C1500'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [108.43700989590789,
  39.10657007181204,
  108.45707249915738,
  39.12609044254127],
 'center': [108.44704119753263, 39.11633025717666, 0]}

In [43]:
# Make a GET request to retrieve information for the date mentioned below
observation_date_2 = items["2023-04-18"]
# Extract collection name and item ID
collection_id = observation_date_2.collection_id
item_id = observation_date_2.id

observation_date_2_tile = requests.get(
    f"{RASTER_API_URL}/collections/{collection_id}/items/{item_id}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map.lower()}"
    f"&rescale=0,1500"
).json()

# Print the properties of the retrieved granule to the console
observation_date_2_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20230418T060602_000822/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=puor_r&rescale=0%2C1500'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [108.43700989590789,
  39.10657007181204,
  108.45707249915738,
  39.12609044254127],
 'center': [108.44704119753263, 39.11633025717666, 0]}

In [44]:
# Set initial zoom and center of map for plume Layer
map_ = folium.Map(location=(observation_date_2_tile["center"][1], observation_date_2_tile["center"][0]), zoom_start=14, tiles=None, tooltip = 'test tool tip')
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}.png', name='ESRI World Imagery', attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',overlay='True').add_to(map_)


# Use the 'TileLayer' library to display the raster layer, add an appropriate caption, and adjust the transparency of the layer on the map
map_layer = TileLayer(
    tiles=observation_date_2_tile["tiles"][0], # Path to retrieve the tile
    name='Plume Complex Landfill',
    overlay='True', # The layer can be overlaid on the map
    attr="GHG", # Set the attribution 
    opacity=1, # Adjust the transparency of the layer
)
map_layer.add_to(map_)


# Adjust map elements 
folium.LayerControl(collapsed=False, position='topright').add_to(map_)

# Add colorbar
# We can use one of 'generate_html_colorbar' from the 'ghgc_utils' module to create an HTML colorbar representation.
legend_html = ghgc_utils.generate_html_colorbar(
                color_map,
                rescale_values,
                label='ppm-m'
    )

# Add colorbar to the map
map_.get_root().html.add_child(folium.Element(legend_html))


# Visualizing the map
map_

## Summary

In this notebook we have successfully completed the following steps for the STAC collection for the EMIT Methane Point Source Plume Complexes dataset:
1.  Install and import the necessary libraries
2.  Fetch the collection from STAC collections using the appropriate endpoints
3.  Count the number of existing granules within the collection
4.  Map the methane emission plumes

If you have any questions regarding this user notebook, please contact us using the [feedback form](https://docs.google.com/forms/d/e/1FAIpQLSeVWCrnca08Gt_qoWYjTo6gnj1BEGL4NCUC9VEiQnXA02gzVQ/viewform).